## 9. Metaprogramming

涉及 装饰器、元类（metaclass）。

目的是不改变已有代码的情况下，去操控代码。 例如装饰器是添加修饰功能，或者完全取代。


又是一个庞杂的东西。

涉及

1. 装饰器
2. metaclass
3. python code执行以及变量（不懂）

###  9.1 wrapper

很简单的装饰器，作为热身

注意在用 `@` 语法时， 后面的对象（可以是函数，可以是类），必须接受一个 函数参数 (func)。

In [17]:
import time
from functools import wraps

def timethis(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n):
    while n > 0:
        n -= 1

countdown(10000)
print(countdown)
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__wrapped__)   ## wraps还给出 __wrapped__属性

countdown 0.0006749629974365234
<function timethis.<locals>.wrapper at 0x10ff42730>
wrapper
None


AttributeError: 'function' object has no attribute '__wrapped__'

### 9.2 使用装饰器时，保留原函数的metadata

注意上面例子中， countdown不再是以前的那个countdown了，已经被掉包了，实际上是一个 wrapper函数对象。

如果要保留原函数的信息，做到以假乱真，需要加上 `@wraps`装饰器

看 `functools.wraps`的源代码就可以发现，就是把 `wrapper` 这个函数的一些metadata信息全部替换掉了。并且 加了一个 `__wrapped__`。


In [18]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n:int):
    while n > 0:
        n -= 1

countdown(10000)
print(countdown)
print(countdown.__name__)
print(countdown.__annotations__)
print(countdown.__wrapped__)   ## wraps还给出 __wrapped__属性

countdown 0.001474142074584961
<function countdown at 0x10ffa16a8>
countdown
{'n': <class 'int'>}
<function countdown at 0x10ff420d0>


### 9.3 Unwrapping a Decorator

假的终归是假的。。还原本来的函数方法。


In [19]:
orig_countdown = countdown.__wrapped__
orig_countdown(1000)
print(orig_countdown)
print(orig_countdown.__name__)
print(orig_countdown.__annotations__)

<function countdown at 0x10ff420d0>
countdown
{'n': <class 'int'>}



这里有个问题，如果有多个装饰器，实际上是一个链状结构

In [42]:
# Example of unwrapping a decorator

from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def add(x, y):
    return x + y

# Calling wrapped function
print(add(2,3))

# Calling original function
print(add.__wrapped__(2,3))

# Calling original function
print(add.__wrapped__.__wrapped__(2,3))

Decorator 1
Decorator 2
5
Decorator 2
5
5


### 9.4 带参数的装饰器

注意不带参数的装饰器对象是一个函数，需要接受一个(func)参数，返回一个 （wrapper）假函数即可。

但是带参数的装饰器，需要先给出参数，然后返回一个不带参数的装饰器对象。

In [23]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    '''
    Add logging to a function.  level is the logging
    level, name is the logger name, and message is the
    log message.  If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)   # 在这里就把所有metadata保留了
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    print(add(2,3))
    spam()

print(add, add.__wrapped__)
print(spam, spam.__wrapped__)

DEBUG:__main__:add
CRITICAL:example:spam


5
Spam!
<function add at 0x10fbc01e0> <function add at 0x10fbc0a60>
<function spam at 0x10fbc0d08> <function spam at 0x10fbc06a8>


### 9.5 装饰器行为运行时调整

其实可以理解成，配置装饰器的属性，可以在运行时修改调整。

进一步理解： 装饰器的作用是替换掉原来的函数。 函数作为一个对象，可以有一些自己的属性。 同时，传进来的参数，是存放到函数堆栈里的。


In [26]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    '''
    Add logging to a function.  level is the logging
    level, name is the logger name, and message is the
    log message.  If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):    # 这就是那个假函数
            log.log(level, logmsg)
            return func(*args, **kwargs)

        # Attach setter functions
        @attach_wrapper(wrapper)    # 对假函数的一些操作
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

# Example involving multiple decorators

import time
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return r
    return wrapper

@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1


@logged(logging.DEBUG)
@timethis
def countdown2(n):
    while n > 0:
        n -= 1

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    print(add(2, 3))

    # Change the log message
    add.set_message('Add called')
    print(add(2, 3))

    # Change the log level
    add.set_level(logging.WARNING)
    print(add(2, 3))

    countdown(100000)
    countdown.set_level(logging.CRITICAL)
    countdown(100000)

    countdown2(100000)
    countdown2.set_level(logging.CRITICAL)
    countdown2(100000)


DEBUG:__main__:add
DEBUG:__main__:Add called
DEBUG:__main__:countdown
CRITICAL:__main__:countdown
DEBUG:__main__:countdown2
CRITICAL:__main__:countdown2


5
5
5
countdown 0.011485815048217773
countdown 0.010819196701049805
countdown2 0.015055179595947266
countdown2 0.02456188201904297


In [44]:
# Alternate formulation using function attributes directly

from functools import wraps
import logging

def logged(level, name=None, message=None):
    '''
    Add logging to a function.  level is the logging
    level, name is the logger name, and message is the
    log message.  If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            wrapper.log.log(wrapper.level, wrapper.logmsg)
            return func(*args, **kwargs)

        # Attach adjustable attributes
        wrapper.level = level
        wrapper.logmsg = logmsg
        wrapper.log = log

        return wrapper
    return decorate

# Example use
@timethis
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.DEBUG)
@timethis
def add2(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    print(add(2, 3))

    # Change the log message
    add.logmsg = 'Add called'
    print(add(2, 3))

    # Change the log level
    add.level = logging.WARNING
    print(add(2, 3))
    
    ## add 2
    print(add2(2, 3))

    # Change the log message
    add2.logmsg = 'Add called'
    print(add2(2, 3))

    # Change the log level
    add2.level = logging.WARNING
    print(add2(2, 3))


DEBUG:__main__:add
DEBUG:__main__:add
DEBUG:__main__:add
DEBUG:__main__:add2
DEBUG:__main__:Add called


add 0.0009200572967529297
5
add 0.0009019374847412109
5
add 0.004828929901123047
5
add2 1.9073486328125e-06
5
add2 1.9073486328125e-06
5
add2 2.1457672119140625e-06
5


In [47]:
print('1. timethis', add.__dict__)
print('1. logged', add.__wrapped__.__dict__)
print('2. logged', add2.__dict__)
print('2. timethis', add2.__wrapped__.__dict__)

1. timethis {'__wrapped__': <function add at 0x10fcfb510>, 'level': 30, 'logmsg': 'Add called', 'log': <Logger __main__ (DEBUG)>}
1. logged {'__wrapped__': <function add at 0x10fcfb400>, 'level': 10, 'logmsg': 'add', 'log': <Logger __main__ (DEBUG)>}
2. logged {'__wrapped__': <function add2 at 0x10fcfbb70>, 'level': 30, 'logmsg': 'Add called', 'log': <Logger __main__ (DEBUG)>}
2. timethis {'__wrapped__': <function add2 at 0x10fcfb840>}


注意以上两种方法的区别

第二种方法有个bug， 如果 装饰器之上再添加一个装饰器，虽然保留这些属性，但是不再对封装的那个装饰器产生影响。

所以相比较而言，添加一个方法（访问器，accessor），比直接添加属性要好。

### 9.6 装饰器，可选参数，更灵活

上面带参数的装饰器和不带参数的装饰器，层数不一样。 其实也可以放到一起。这里需要用到 位置参数的限定，即不允许使用位置参数。

In [50]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

# Example use
@logged         # 其实是不带参数的装饰器，传递了一个func：add进来。
def add(x, y):
    return x + y

@logged()       # 是带参数的装饰器，但是 func =None， 所以用partial打个包，变成不带参数的装饰器
def sub(x, y):
    return x - y

@logged(level=logging.CRITICAL, name='example')   # 同上，func=None
def spam():     
    print('Spam!')

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    add(2,3)
    sub(2,3)
    spam()


DEBUG:__main__:add
DEBUG:__main__:sub
CRITICAL:example:spam


Spam!


### 9.7 使用装饰器做类型检查

之前见过，一种操作。

In [51]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func

        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                            )
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Examples

@typeassert(int, int)
def add(x, y):
    return x + y

@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

if __name__ == '__main__':
    print(add(2,3))
    try:
        add(2, 'hello')
    except TypeError as e:
        print(e)

    spam(1, 2, 3)
    spam(1, 'hello', 3)
    try:
        spam(1, 'hello', 'world')
    except TypeError as e:
        print(e)



5
Argument y must be <class 'int'>
1 2 3
1 hello 3
Argument z must be <class 'int'>



### 9.8 类方法作为装饰器

其实 property 就用了一些方法作为装饰器

In [52]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper

    # Decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper

# Example
# As an instance method
a = A()

@a.decorator1
def spam():
    pass

# As a class method
@A.decorator2
def grok():
    pass

spam()
grok()


Decorator 1
Decorator 2


In [53]:
# Property example

class Person:
    first_name = property()
    @first_name.getter
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

p = Person()
p.first_name = 'Dave'
print(p.first_name)


Dave


### 9.9 类作为装饰器

其实跟函数装饰器一样， 类的 `__init__` 方法需要接受一个 `func` 参数。

然后替换掉的不再是一个函数，实际上是一个类装饰器的一个实例。

In [68]:
# Example of defining a decorator as a class
import types
from functools import wraps
       
class Profiled:
    def __init__(self, func):
        wraps(func)(self)    # 这里直接更新一个 metadata，假装自己就是原函数。
        self.ncalls = 0

    def __call__(self, *args, **kwargs):
        self.ncalls += 1    # 每次调用计数， 这是装饰器唯一添加的功能
        return self.__wrapped__(*args, **kwargs)   # 调用原函数

    def __get__(self, instance, cls):   # ？？？？？
        print('get!!')
        print(instance, cls)
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)

# Example

@Profiled
def add(x, y):
    return x + y

class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)

if __name__ == '__main__':
    print(add(2,3))
    print(add(4,5))
    print('ncalls:', add.ncalls)

    s = Spam()
    s.bar(1)
    s.bar(2)
    s.bar(3)
    print('ncalls:', Spam.bar.ncalls)


5
9
ncalls: 2
get!!
<__main__.Spam object at 0x10fcc0780> <class '__main__.Spam'>
<__main__.Spam object at 0x10fcc0780> 1
get!!
<__main__.Spam object at 0x10fcc0780> <class '__main__.Spam'>
<__main__.Spam object at 0x10fcc0780> 2
get!!
<__main__.Spam object at 0x10fcc0780> <class '__main__.Spam'>
<__main__.Spam object at 0x10fcc0780> 3
get!!
None <class '__main__.Spam'>
ncalls: 3


注意类里面定义的 `__get__` 方法。 这么做的原因是这样的： Spam的bar已经不是原来的那个bar了， 而是一个 Profiled 实例。 这个实例还没有跟 Spam类的实例 bond在一起。 所以定义这么一个方法，用于绑定调用者（instance）与自己(self)。 见下例

In [69]:
s = Spam()
def grok(self, x):
    pass

grok.__get__(s, Spam)  # 第二个参数有啥用？

<bound method grok of <__main__.Spam object at 0x10fcc0c18>>

### 9.10 类方法和静态方法装饰器

`@classmethod`

`@staticmethod`

下面例子中，一定要注意装饰器的顺序。 @classmethod 一定是最上层的装饰器。否则就出错了。 涉及到boud一些问题？？？？

In [71]:
import time
from functools import wraps

# A simple decorator
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(end-start)
        return r
    return wrapper

# Class illustrating application of the decorator to different kinds of methods
class Spam:
    @timethis
    def instance_method(self, n):
        print(self, n)
        while n > 0:
            n -= 1

    @classmethod
    @timethis
    def class_method(cls, n):
        print(cls, n)
        while n > 0:
            n -= 1

    @staticmethod
    @timethis
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1

if __name__ == '__main__':
    s = Spam()
    s.instance_method(1000000)
    Spam.class_method(1000000)
    Spam.static_method(1000000)


<__main__.Spam object at 0x10fc06710> 1000000
0.10002517700195312
<class '__main__.Spam'> 1000000
0.07831406593322754
1000000
0.07678532600402832


### 9.11 给原函数增加一些参数



In [72]:

from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)
    return wrapper

@optional_debug
def spam(a, b, c):
    print(a, b, c)
    
spam(1, 2, 3)
spam(1, 2, 3, debug=True)

1 2 3
Calling spam
1 2 3



### 9.12 类的装饰器（Patch）

一种邪恶的方法，去修改类内部的一些行为。 表面上不去修改类。 本质上，仍然是修改类。

有时候叫 monkeypatching。好像在哪见过， 似乎是 flask还是哪里，用 协程替代线程。

In [73]:
def log_getattribute(cls):
    # Get the original implementation
    orig_getattribute = cls.__getattribute__

    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)

    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls

# Example use
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass

if __name__ == '__main__':
    a = A(42)
    print(a.x)
    a.spam()


getting: x
42
getting: spam


### 9.13 Metaclass ，控制 实例创建

最典型的应用是 单例模式（singleton），见下面例子


关于 metaclass的知识， 这片 <https://jakevdp.github.io/blog/2012/12/01/a-primer-on-python-metaclasses/> 介绍的很好。

简而言之， metaclass 实际上是 type的子类。

一般定义类，实际上都是  `type(name, parents, dict)` 这样的。

如果定一个 Metaclass， 实际上是 `Metaclass(name, parents, dict)` 这样的。 在metaclass里面，会添加类的一些特别的用法。


关于 metaclass，有一段话：

> Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why).

>– Tim Peters

意思是 99% 都用不到这个。 用的时候自然有用的原因。

In [92]:
class Singleton(type):
    def __init__(self, *args, **kwargs):   # 这是创建一个类的方法。可以这么说， MetaClass的实例就是Class。 Class的类型就是 MetaClass
        self.__instance = None
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):   # 这是类调用（一般就是创建实例）的方法。 
        print('call')
        print(self)
        print(super())
        print(super().__call__)
        if self.__instance is None:
#             self.__instance = super().__call__(*args, **kwargs)  # 这里的super，感觉是调用了Spam，但是为什么？
            self.__instance = type.__call__(self, *args, **kwargs) 
            return self.__instance
        else:
            return self.__instance

class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')

if __name__ == '__main__':
    a = Spam()
    b = Spam()
    print(a is b)


call
<class '__main__.Spam'>
<super: <class 'Singleton'>, <Singleton object>>
<method-wrapper '__call__' of Singleton object at 0x7f974eea7258>
Creating Spam
call
<class '__main__.Spam'>
<super: <class 'Singleton'>, <Singleton object>>
<method-wrapper '__call__' of Singleton object at 0x7f974eea7258>
True


上面一直的一个困惑是 `self.__instance = super().__call__(*args, **kwargs)` ， 我知道其作用是创建一个实例。但是用到了 `super()`，就搞不明白了。 实际上， 就是用 metaclass的父类 type方法。

这里有一个回答解释的比较合理： <https://stackoverflow.com/questions/6966772/using-the-call-method-of-a-metaclass-instead-of-new/6966942#6966942>

其他参考资料：

- <https://eli.thegreenplace.net/2011/08/14/python-metaclasses-by-example#id16>

原因是这样子的：
```python
class type:
    def __call__(cls, *args, **kwarg):

        # ... a few things could possibly be done to cls here... maybe... or maybe not...

        # then we call cls.__new__() to get a new object
        obj = cls.__new__(cls, *args, **kwargs)

        # ... a few things done to obj here... maybe... or not...

        # then we call obj.__init__()
        obj.__init__(*args, **kwargs)

        # ... maybe a few more things done to obj here

        # then we return obj
        return obj
```

可以稍微理解 type的 `__call__` 干了什么事情。在这里调用了方法



In [85]:
print(super(Singleton, Spam))

<super: <class 'Singleton'>, <Singleton object>>


In [86]:
print(Singleton.__mro__)

(<class '__main__.Singleton'>, <class 'type'>, <class 'object'>)


In [87]:
print(Spam.__mro__)

(<class '__main__.Spam'>, <class 'object'>)


In [89]:
print(type(Spam))
print(type(Singleton))

<class '__main__.Singleton'>
<class 'type'>


In [80]:
print(a)

1


为了说明这一点，注意下面这个例子， 这里更能看出来。 Spam不能实例化，原因是在 metaclass的 `__call__` 里禁用了这一点。

In [93]:
class NoInstances(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")

class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

if __name__ == '__main__':
    try:
        s = Spam()
    except TypeError as e:
        print(e)

    Spam.grok(42)


Can't instantiate directly
Spam.grok



### 9.14 metaclass的一个例子： 保存 属性定义的顺序

注意用一个顺序字典就可以了。 但是这个应用在哪里不是很清楚

下面这个例子的重点其实就是 `__prepare__` 方法。

参考 python的 [data model](https://docs.python.org/3/reference/datamodel.html)

In [94]:
# Example of capturing class definition order

from collections import OrderedDict

# A set of descriptors for various types
class Typed:
    _expected_type = type(None)
    def __init__(self, name=None):
        self._name = name

    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' +str(self._expected_type))
        instance.__dict__[self._name] = value

class Integer(Typed):
    _expected_type = int

class Float(Typed):
    _expected_type = float

class String(Typed):
    _expected_type = str

# Metaclass that uses an OrderedDict for class body
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        order = []
        for name, value in clsdict.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)

    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()

# Example class that uses the definition order to initialize members
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)

# Example use
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

if __name__ == '__main__':
    s = Stock('GOOG',100,490.1)
    print(s.name)
    print(s.as_csv())
    try:
        t = Stock('AAPL','a lot', 610.23)
    except TypeError as e:
        print(e)



GOOG
GOOG,100,490.1
Expected <class 'int'>


### 9.15 带optional参数 metaclass

metaclass不是重点，以后再说

In [95]:
# Example of a metaclass that takes optional arguments

class MyMeta(type):
    # Optional
    @classmethod
    def __prepare__(cls, name, bases, *, debug=False, synchronize=False):
        # Custom processing
        return super().__prepare__(name, bases)

    # Required
    def __new__(cls, name, bases, ns, *, debug=False, synchronize=False):
        # Custom processing
        return super().__new__(cls, name, bases, ns)
        
    def __init__(self, name, bases, ns, *, debug=False, synchronize=False):
        # Custom processing
        super().__init__(name, bases, ns)

# Examples
class A(metaclass=MyMeta, debug=True, synchronize=True):
    pass

class B(metaclass=MyMeta):
    pass

class C(metaclass=MyMeta, synchronize=True):
    pass

### 9.16 方法签名检查 `*args, **kwargs`

In [97]:
# Example of code that enforces signatures on an __init__ function

from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
             for name in names]
    return Signature(parms)

class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

# Example use
class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')

class Point(Structure):
    __signature__ = make_sig('x', 'y')

# Example instantiation tests
if __name__ == '__main__':
    s1 = Stock('ACME', 100, 490.1)
    print(s1.name, s1.shares, s1.price)

    s2 = Stock(shares=100, name='ACME', price=490.1)
    print(s2.name, s2.shares, s2.price)

    # Not enough args
    try:
        s3 = Stock('ACME', 100)
    except TypeError as e:
        print(e)

    # Too many args
    try:
        s4 = Stock('ACME', 100, 490.1, '12/21/2012')
    except TypeError as e:
        print(e)

    # Replicated args
    try:
        s5 = Stock('ACME', 100, name='ACME', price=490.1)
    except TypeError as e:
        print(e)


ACME 100 490.1
ACME 100 490.1
missing a required argument: 'price'
too many positional arguments
multiple values for argument 'name'


### 9.17 类里面强制 coding风格

似乎没什么必要。。。。用处不多。 但凡 metaclass都用处不多。 

In [98]:
class NoMixedCaseMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        for name in clsdict:
            if name.lower() != name:
                raise TypeError('Bad attribute name: ' + name)
        return super().__new__(cls, clsname, bases, clsdict)

class Root(metaclass=NoMixedCaseMeta):
    pass

class A(Root):
    def foo_bar(self):      # Ok
        pass

print('**** About to generate a TypeError')
class B(Root):
    def fooBar(self):       # TypeError
        pass


**** About to generate a TypeError


TypeError: Bad attribute name: fooBar

### 9.18 手动创建类

其实就是用 `type(classname, base, namespace)` 来创建

In [99]:
# Methods
def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price

def cost(self):
    return self.shares * self.price

cls_dict = {
    '__init__' : __init__,
    'cost' : cost,
}

# Make a class
import types

Stock = types.new_class('Stock', (), {}, lambda ns: ns.update(cls_dict))

if __name__ == '__main__':
    s = Stock('ACME', 50, 91.1)
    print(s)
    print(s.cost())


4555.0


### 9.19 在定义时就创建类成员

又是一个黑科技。

下面这个例子主要注意两点

1. metaclass里的 `__init__(cls, *args, **kwargs)`。 黑科技之创建类
2. `property(operator.itemgetter(n))` 黑科技之 `itemgtter`

In [100]:
import operator

class StructTupleMeta(type):
    def __init__(cls, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for n, name in enumerate(cls._fields_):
            setattr(cls, name, property(operator.itemgetter(n)))

class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields_ = []
    def __new__(cls, *args):
        if len(args) != len(cls._fields_):
            raise ValueError('{} arguments required'.format(len(cls._fields_)))
        return super().__new__(cls,args)

# Examples
class Stock(StructTuple):
    _fields_ = ['name', 'shares', 'price']

class Point(StructTuple):
    _fields_ = ['x', 'y']

if __name__ == '__main__':
    s = Stock('ACME', 50, 91.1)
    print(s)
    print(s[0])
    print(s.name)
    print(s.shares * s.price)
    try:
        s.shares = 23
    except AttributeError as e:
        print(e)


('ACME', 50, 91.1)
ACME
ACME
4555.0
can't set attribute



### 9.20 方法多态（不同签名）

In [101]:
import inspect
import types

class MultiMethod:
    '''
    Represents a single multimethod.
    '''
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name

    def register(self, meth):
        '''
        Register a new method as a multimethod
        '''
        sig = inspect.signature(meth)

        # Build a type-signature from the method's annotations
        types = []
        for name, parm in sig.parameters.items():
            if name == 'self': 
                continue
            if parm.annotation is inspect.Parameter.empty:
                raise TypeError(
                    'Argument {} must be annotated with a type'.format(name)
                    )
            if not isinstance(parm.annotation, type):
                raise TypeError(
                    'Argument {} annotation must be a type'.format(name)
                    )
            if parm.default is not inspect.Parameter.empty:
                self._methods[tuple(types)] = meth
            types.append(parm.annotation)

        self._methods[tuple(types)] = meth

    def __call__(self, *args):
        '''
        Call a method based on type signature of the arguments
        '''
        types = tuple(type(arg) for arg in args[1:])
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            raise TypeError('No matching method for types {}'.format(types))
        
    def __get__(self, instance, cls):
        '''
        Descriptor method needed to make calls work in a class
        '''
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self
    
class MultiDict(dict):
    '''
    Special dictionary to build multimethods in a metaclass
    '''
    def __setitem__(self, key, value):
        if key in self:
            # If key already exists, it must be a multimethod or callable
            current_value = self[key]
            if isinstance(current_value, MultiMethod):
                current_value.register(value)
            else:
                mvalue = MultiMethod(key)
                mvalue.register(current_value)
                mvalue.register(value)
                super().__setitem__(key, mvalue)
        else:
            super().__setitem__(key, value)

class MultipleMeta(type):
    '''
    Metaclass that allows multiple dispatch of methods
    '''
    def __new__(cls, clsname, bases, clsdict):
        return type.__new__(cls, clsname, bases, dict(clsdict))

    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()


# Some example classes that use multiple dispatch
class Spam(metaclass=MultipleMeta):
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)
    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)

# Example: overloaded __init__
import time
class Date(metaclass=MultipleMeta):
    def __init__(self, year: int, month:int, day:int):
        self.year = year
        self.month = month
        self.day = day

    def __init__(self):
        t = time.localtime()
        self.__init__(t.tm_year, t.tm_mon, t.tm_mday)

if __name__ == '__main__':
    s = Spam()
    s.bar(2, 3)
    s.bar('hello')
    s.bar('hello', 5)
    try:
        s.bar(2, 'hello')
    except TypeError as e:
        print(e)

    # Overloaded __init__
    d = Date(2012, 12, 21)
    print(d.year, d.month, d.day)
    # Get today's date
    e = Date()
    print(e.year, e.month, e.day)


Bar 1: 2 3
Bar 2: hello 0
Bar 2: hello 5
No matching method for types (<class 'int'>, <class 'str'>)
2012 12 21
2018 4 1


### 9.21 节省代码的黑科技之 ：属性定义

In [102]:
def typed_property(name, expected_type):
    storage_name = '_' + name

    @property
    def prop(self):
        return getattr(self, storage_name)

    @prop.setter
    def prop(self, value):
        if not isinstance(value, expected_type):
            raise TypeError('{} must be a {}'.format(name, expected_type))
        setattr(self, storage_name, value)
    return prop

# Example use
class Person:
    name = typed_property('name', str)
    age = typed_property('age', int)
    def __init__(self, name, age):
        self.name = name
        self.age = age

if __name__ == '__main__':
    p = Person('Dave', 39)
    p.name = 'Guido'
    try:
        p.age = 'Old'
    except TypeError as e:
        print(e)


age must be a <class 'int'>


### 9.22 节省代码黑科技之： Context Manager

注意下面这个例子， 用了一个装饰器 `contextmanager`。

理论上，这个装饰器可以给新的`timethis` 对象，添加 `__enter__` 和 `__exit__` 方法。  在使用的时候，进入开始计时，出去的时候结算。

参考 <https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/> 很好的描述了这一点。（前面还有很多讲 file handle重要性的东西）。

`contextmanager` 装饰的是一个 generator！！！并且只 yield 一次！！！，重要的事情说一遍！！！。 在yield之前的都是 `__enter__` 要做的； 在yield之后 的都是 `__exit__` 要做的。

具体实现的描述： 在 `__enter__` 里， 返回 `next(gen)`， 所以会运行 `yield` 之前的代码，并且返回 `yield` 的东西。

在 `__exit__` 里， 又一个黑科技。。。。`self.gen.throw`，似乎是退到了 `timethis` 的 `finally` 中。。。继续执行。。。

In [ ]:
import time
from contextlib import contextmanager

@contextmanager
def timethis(label):
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print('{}: {}'.format(label, end - start))

# Example use
with timethis('counting'):
    n = 10000000
    while n > 0:
        n -= 1


In [103]:
from contextlib import contextmanager

@contextmanager
def list_transaction(orig_list):
    working = list(orig_list)
    yield working
    orig_list[:] = working

# Example
if __name__ == '__main__':
    items = [1, 2, 3]
    with list_transaction(items) as working:
        working.append(4)
        working.append(5)
    print(items)
    try:
        with list_transaction(items) as working:
            working.append(6)
            working.append(7)
            raise RuntimeError('oops')
    except RuntimeError as e:
        print(e)

    print(items)


[1, 2, 3, 4, 5]
oops
[1, 2, 3, 4, 5]


这第二个例子，就说明了 context 的重要性。 在操作中如果出现异常， 就不会进入 `__exit__` 环节，所以不影响结果。

### 9.23 执行 code， local 变量

在调用 `exec` 时的一个问题。 关于 `exec` 参考 <http://lucumr.pocoo.org/2011/2/1/exec-in-python/>

关于 `locals()` 又是一个故事。

参考 <https://stackoverflow.com/questions/7969949/whats-the-difference-between-globals-locals-and-vars>

注意 locals的范围。

In [111]:
try:
    del b
except:
    pass

def test():
    a = 13
    exec('b = a + 1')
    print(b)      # --> 14

test()

NameError: name 'b' is not defined

注意找不到 b。

In [115]:
def test():
    a = 13
    loc = locals()
    exec('b = a + 1')
    print(loc)
    b = loc['b']
    print(b)      # --> 14
test()

{'a': 13, 'loc': {...}, 'b': 14}
14


In [120]:
def test1():
    x = 0
    exec('x += 1')
    print(x)      # --> 0
test1()
print()

def test2():
    x = 0
    loc = locals()
    print('before:', loc)
    exec('x += 1')
    print('after:', loc)
    print('x =', x)
test2()
print()

def test3():
    x = 0
    loc = locals()
    print(loc)
    exec('x += 1')
    print(loc)
    locals()
    print(loc)
test3()
print()

0

before: {'x': 0}
after: {'x': 1, 'loc': {...}}
x = 0

{'x': 0}
{'x': 1, 'loc': {...}}
{'x': 0, 'loc': {...}}




### _TODO: 9.24 解析和执行python code（在python代码中）。。_


其实理解这个，就能理解 python code是怎么执行的。 

In [126]:
import ast

class CodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.loaded = set()
        self.stored = set()
        self.deleted = set()
    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Load):
            self.loaded.add(node.id)
        elif isinstance(node.ctx, ast.Store):
            self.stored.add(node.id)
        elif isinstance(node.ctx, ast.Del):
            self.deleted.add(node.id)

# Sample usage
if __name__ == '__main__':
    # Some python code
    code = '''
for i in range(10): 
    print(i)
del i
'''
    # Parse into an AST
    top = ast.parse(code, mode='exec')

    # Feed the AST to analyze name usage
    c = CodeAnalyzer()
    c.visit(top)
    print('Loaded:', c.loaded)
    print('Stored:', c.stored)
    print('Deleted:', c.deleted)


Loaded: {'print', 'range', 'i'}
Stored: {'i'}
Deleted: {'i'}


In [128]:
# namelower.py
import ast
import inspect

# Node visitor that lowers globally accessed names into
# the function body as local variables. 
class NameLower(ast.NodeVisitor):
    def __init__(self, lowered_names):
        self.lowered_names = lowered_names

    def visit_FunctionDef(self, node):
        # Compile some assignments to lower the constants
        code = '__globals = globals()\n'
        code += '\n'.join("{0} = __globals['{0}']".format(name)
                          for name in self.lowered_names)

        code_ast = ast.parse(code, mode='exec')

        # Inject new statements into the function body
        node.body[:0] = code_ast.body

        # Save the function object
        self.func = node

# Decorator that turns global names into locals
def lower_names(*namelist):
    def lower(func):
        srclines = inspect.getsource(func).splitlines()
        # Skip source lines prior to the @lower_names decorator
        for n, line in enumerate(srclines):
            if '@lower_names' in line:
                break

        src = '\n'.join(srclines[n+1:])
        # Hack to deal with indented code
        if src.startswith((' ','\t')):
            src = 'if 1:\n' + src
        top = ast.parse(src, mode='exec')

        # Transform the AST 
        cl = NameLower(namelist)
        cl.visit(top)

        # Execute the modified AST
        temp = {}
        exec(compile(top,'','exec'), temp, temp)

        # Pull out the modified code object
        func.__code__ = temp[func.__name__].__code__
        return func
    return lower

# Example of use
INCR = 1

def countdown1(n):
    while n > 0:
        n -= INCR

@lower_names('INCR')
def countdown2(n):
    while n > 0:
        n -= INCR

if __name__ == '__main__':
    import time
    print('Running a performance check')

    start = time.time()
    countdown1(1000000)
    end = time.time()
    print('countdown1:', end-start)

    start = time.time()
    countdown2(1000000)
    end = time.time()
    print('countdown2:', end-start)



Running a performance check
countdown1: 0.12802362442016602
countdown2: 0.12523603439331055


### 9.25 Disassembling Python Byte Code

In [129]:
# Example of manual disassembly of bytecode

import opcode

def generate_opcodes(codebytes):
    extended_arg = 0
    i = 0
    n = len(codebytes)
    while i < n:
        op = codebytes[i]
        i += 1
        if op >= opcode.HAVE_ARGUMENT:
            oparg = codebytes[i] + codebytes[i+1]*256 + extended_arg
            extended_arg = 0
            i += 2
            if op == opcode.EXTENDED_ARG:
                extended_arg = oparg * 65536
                continue
        else:
            oparg = None
        yield (op, oparg)

# Example
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
    print('Blastoff!')

for op, oparg in generate_opcodes(countdown.__code__.co_code):
    print(op, opcode.opname[op], oparg)


120 SETUP_LOOP 31774
0 <0> None
100 LOAD_CONST 27393
4 DUP_TOP None
114 POP_JUMP_IF_FALSE 29726
0 <0> None
100 LOAD_CONST 31746
0 <0> None
131 CALL_FUNCTION 258
0 <0> None
124 LOAD_FAST 25600
3 ROT_THREE None
56 INPLACE_SUBTRACT None
0 <0> None
125 STORE_FAST 28928
2 ROT_TWO None
87 POP_BLOCK None
0 <0> None
116 LOAD_GLOBAL 25600
4 DUP_TOP None
131 CALL_FUNCTION 257
0 <0> None
100 LOAD_CONST 21248
0 <0> None


In [130]:
import dis

dis.dis(countdown)

 25           0 SETUP_LOOP              30 (to 32)
        >>    2 LOAD_FAST                0 (n)
              4 LOAD_CONST               1 (0)
              6 COMPARE_OP               4 (>)
              8 POP_JUMP_IF_FALSE       30

 26          10 LOAD_GLOBAL              0 (print)
             12 LOAD_CONST               2 ('T-minus')
             14 LOAD_FAST                0 (n)
             16 CALL_FUNCTION            2
             18 POP_TOP

 27          20 LOAD_FAST                0 (n)
             22 LOAD_CONST               3 (1)
             24 INPLACE_SUBTRACT
             26 STORE_FAST               0 (n)
             28 JUMP_ABSOLUTE            2
        >>   30 POP_BLOCK

 28     >>   32 LOAD_GLOBAL              0 (print)
             34 LOAD_CONST               4 ('Blastoff!')
             36 CALL_FUNCTION            1
             38 POP_TOP
             40 LOAD_CONST               0 (None)
             42 RETURN_VALUE


In [132]:
countdown.__code__.co_code

b'x\x1e|\x00d\x01k\x04r\x1et\x00d\x02|\x00\x83\x02\x01\x00|\x00d\x038\x00}\x00q\x02W\x00t\x00d\x04\x83\x01\x01\x00d\x00S\x00'